In [ ]:
import sys
import numpy as np
import cv2


In [ ]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('no')
    sys.exit()

dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('no')
    sys.exit()

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0 , 255).astype(np.uint8)

dst = np.zeros(src.shape[:2], np.uint8)
_, dst = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('amg', mag)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('no')
    sys.exit()

dst= cv2.Canny(src, 50,150)
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()


In [ ]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('no')
    sys.exit()

edges = cv2.Canny(src, 50,150)

lines = cv2.HoughLinesP(edges, 1, np.pi/180. , 160, minLineLength=50, maxLineGap=5)
dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0,0,255),2,cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [29]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/dial.jpg')

if src is None:
    print('no')
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0,0), 1.0)

def on_trackbar(pos):
    rmin = cv2.getTrackbarPos('minRadius', 'img')
    rmax = cv2.getTrackbarPos('maxRadius', 'img')
    th = cv2.getTrackbarPos('threshold', 'img')

    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50, param1=120, param2=th, minRadius=rmin, maxRadius=rmax)
    dst = src.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = circles[0][i]
            print(cx,cy,radius)
            cv2.circle(dst, (int(cx),int(cy)), int(radius), (0,0,255), 2, cv2.LINE_AA)

    cv2.imshow('img', dst)

cv2.imshow('img', src)
cv2.createTrackbar('minRadius', 'img', 0, 100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0, 150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 0, 100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)

cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/hough.cpp:2282: error: (-211:One of the arguments' values is out of range) acc_threshold must be a positive number in function 'HoughCircles'


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/hough.cpp:2282: error: (-211:One of the arguments' values is out of range) acc_threshold must be a positive number in function 'HoughCircles'


341.5 666.5 43.8
452.5 663.5 43.5
325.5 266.5 53.0
530.5 313.5 49.4
236.5 324.5 46.3
544.5 605.5 41.6
182.5 413.5 43.8
434.5 265.5 53.0
245.5 615.5 46.3
185.5 518.5 43.0
115.5 228.5 13.4
42.5 577.5 14.2


In [ ]:
src = cv2.imread('/home/addinedu/dev_ws/opencv/data/coins1.jpg')

if src is None:
    print('no')
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0,0), 1)
circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50, param1=150, param2=40, minRadius=20, maxRadius=80)

sum_of_money = 0
dst = src.copy()

if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (int(cx),int(cy)), int(radius), (0,0,255), 2, cv2.LINE_AA)

        x1 = int(cx - radius)
        y1 = int(cy - radius)
        x2 = int(cx + radius)
        y2 = int(cy + radius)
        radius = int(radius)

        crop = dst[y1:y2, x1:x2, :]
        ch, cw = crop.shape[:2]

        mask = np.zeros((ch, cw), np.uint8)
        cv2.circle(mask, (int(cw//2), int(ch//2)), int(radius),255,-1)

        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        hue, _, _ = cv2.split(hsv)
        hue_shift = (hue+40)%180
        mean_of_hue = cv2.mean(hue_shift, mask)[0]

        won = 100
        if mean_of_hue < 52:
            won = 10

        sum_of_money += won

        # cv2.putText(crop, str(won), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2, cv2.LINE_AA)
        cv2.putText(crop, str(won), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2, cv2.LINE_AA)

cv2.putText(dst, str(sum_of_money) + 'won', (40,80), cv2.FONT_HERSHEY_DUPLEX, 2, (255,0,0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()